In [1]:
import simpleFoam_tools as sft
import kphi
import pandas as pd
import numpy as np

In [2]:
remove = 1
if remove:
    sft.remove_run_files(".")

In [3]:
domain_name = "cylinder"
vti_path = f"constant/geometry/{domain_name}.vti"
stl_path = f"constant/triSurface/{domain_name}.stl"

In [4]:
radii = 10
shape = (100, 50, 50)

In [5]:
sft.make_cylinder_vti(radii, shape[::-1], vti_path)

✅ Modified and saved: constant/geometry/cylinder_raw.vti
✅ Modified and saved: constant/geometry/cylinder.vti
Created VTI file: constant/geometry/cylinder.vti


In [6]:
scale=1e-6
dp = 1e-3
stl = f"{domain_name}.stl"
refinement = 1
factor_mesh = 1
location_in_mesh = (25, 25, 25)
# location_in_mesh = (50, 70, 20)
boundary_type = "Wall" # "Wall" or "symmetryPlane"
end_time = 300
write_interval = 50

In [7]:
sft.make_stl(vti_path, stl_path)

Running command: pvpython /home/h09435ap/porePermFoam/simpleFoam_tools/paraview_stl.py /home/h09435ap/porePermFoam/constant/geometry/cylinder.vti /home/h09435ap/porePermFoam/constant/triSurface/cylinder.stl
hwloc/linux: Ignoring PCI device with non-16bit domain.
Pass --enable-32bits-pci-domain to configure to support such devices
(warning: it would break the library ABI, don't enable unless really needed).
✅ Wrote ASCII STL: /home/h09435ap/porePermFoam/constant/triSurface/cylinder.stl


In [8]:
mesh_resolution = tuple(x * factor_mesh for x in shape)
blockMeshDict_path    = "system/blockMeshDict"
controlDict_path      = "system/controlDict"
p_field_path          = "0/p"
U_field_path          = "0/U"
snappyHexMeshDict_path = "system/snappyHexMeshDict"
sft.generate_blockMeshDict(shape, mesh_resolution, blockMeshDict_path, boundary=boundary_type)
sft.generate_snappyHexMeshDict(location_in_mesh, stl, snappyHexMeshDict_path, refinement=refinement)
sft.generate_controlDict(controlDict_path, end_time=end_time, write_interval=write_interval)
sft.generate_pressure_field(p_field_path, dp=dp, boundary=boundary_type)
sft.generate_velocity_field(U_field_path, boundary=boundary_type)

Generated blockMeshDict at: system/blockMeshDict
Generated snappyHexMeshDict at: system/snappyHexMeshDict
Generated controlDict at: system/controlDict
Generated p at: 0/p with boundary zeroGradient
Generated U at: 0/U with boundary noSlip


In [9]:
sft.run_simplefoam(".", scale=scale)


>>> blockMesh

/*---------------------------------------------------------------------------*\
| =========                 |                                                 |
| \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox           |
|  \\    /   O peration     | Version:  2412                                  |
|   \\  /    A nd           | Website:  www.openfoam.com                      |
|    \\/     M anipulation  |                                                 |
\*---------------------------------------------------------------------------*/
Build  : _f00230df-20250528 OPENFOAM=2412 patch=250528 version=2412
Arch   : "LSB;label=32;scalar=64"
Exec   : blockMesh
Date   : Aug 06 2025
Time   : 21:37:23
Host   : e-10aux32876g5
PID    : 1834948
I/O    : uncollated
Case   : /home/h09435ap/porePermFoam
nProcs : 1
trapFpe: Floating point exception trapping enabled (FOAM_SIGFPE).
fileModificationChecking : Monitoring run-time modified files using timeStampMaster (file

In [10]:
df_q = pd.read_csv("q_in.csv")
df_q.head()

,time,flowRate_phi,area
0,0.0,-4.397770e-14,3.457440e-10
1,50.0,-4.568970e-14,3.457440e-10
2,100.0,-4.575490e-14,3.457440e-10
3,150.0,-4.575730e-14,3.457440e-10
4,200.0,-4.575740e-14,3.457440e-10


In [11]:
phi = kphi.phi_vti(vti_path)
print(f"porosity: {phi:.02%}")

porosity: 22.28%


In [12]:
A = shape[1] * shape[2] * scale**2
miu = 1e-3
L = shape[0] * scale
Q = np.abs(df_q["flowRate_phi"].iloc[-1])  
dp = dp * 1e3 # convert to Pa
k = Q * miu * L / (A * dp)
print(f"permeability: {k:.02e} m^2")
print(f"permeability: {k * 1.01324997e15:.02f} md")

permeability: 1.83e-12 m^2
permeability: 1854.55 md


In [13]:
k_analytical = kphi.k_cylinder(radii*scale)
A_tube = np.pi * (radii * scale)**2
k_analytical = k_analytical * A_tube / A
print(f"analytical permeability: {k_analytical:.02e} m^2")
print(f"analytical permeability: {k_analytical * 1.01324997e15:.02f} md")
print(f"relative error: {(k - k_analytical) / k_analytical:.02%}")

analytical permeability: 1.57e-12 m^2
analytical permeability: 1591.61 md
relative error: 16.52%


In [14]:
A_mesh = df_q["area"].iloc[-1]
rel_error = np.abs(A_tube - A_mesh) / A_tube
print(f"mesh area: {A_mesh:.02e} m^2")
print(f"analytical area: {A_tube:.02e} m^2")
print(f"relative error in mesh area: {rel_error:.02%}")

mesh area: 3.46e-10 m^2
analytical area: 3.14e-10 m^2
relative error in mesh area: 10.05%
